- This code has written by Ahmadreza Attarpour (a.attarpour@mail.utoronto.ca)
- It's a practice to check how we can use LangChain/LangGraph to build tools and agents

-------------------------------------

# ReAct Agent from Scratch Using Python

Agent --> Think --> Action (LLM or Tool call) --> observation --> repeat


In [1]:
import os

In [2]:
# load the environment variables from the .env file
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# Read the variable from the environment and set it as a env variable
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")
if GOOGLE_API_KEY:
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
if LANGCHAIN_API_KEY:
    os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
if LANGCHAIN_PROJECT:
    os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT
if os.getenv("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
if os.getenv("SERPER_API_KEY"):
    os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

In [ ]:
# # load the LLM and embeddings from langchain_google_genai
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
# embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
# from langchain_google_genai import ChatGoogleGenerativeAI
# llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro")

In [4]:
# from langchain_huggingface import HuggingFaceEmbeddings
# This is a sentence-transformers model: It maps sentences & paragraphs to a
# 384 dimensional dense vector space and can be used for tasks like clustering or semantic search.
# embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
from langchain_groq import ChatGroq
# llm = ChatGroq(model="Gemma2-9b-It")
# llm = ChatGroq(model="llama3-70b-8192")

In [5]:
llm.invoke("Hello, who won the Nobel Prize recently?")

AIMessage(content="The Nobel Prizes are awarded annually, and the winners are announced in October of each year. Here are the most recent winners of the Nobel Prizes:\n\n**2022 Nobel Prize Winners:**\n\n1. **Physics:** Alain Aspect, John F. Clauser, and Anton Zeilinger for their experiments on entangled photons, which have paved the way for more secure quantum communication.\n2. **Chemistry:** Carolyn R. Bertozzi, Morten Meldal, and Barry Sharpless for the development of click chemistry and bioorthogonal chemistry, which allow for the creation of new molecules and the study of biological processes.\n3. **Medicine:** Svante Pääbo for his discoveries concerning the genomes of extinct hominins and the origins of modern humans.\n4. **Literature:** Annie Ernaux for the courage and clinical acuity with which she uncovers the roots, alienations, and collective restraints of personal memory.\n5. **Peace:** Ales Bialiatski, Memorial, and the Center for Civil Liberties for their outstanding effo

# 1. we need a Chatbot class that needs to have three functions: init, call, and execute

In [5]:
class Chatbot:
    """A simple chatbot class that uses a language model to respond to user messages."""

    def __init__(self, system=""):
        """
        Initializes the chatbot with an optional system message.
        Args:
            system (str): An optional system message to set the context for the chatbot.
        """
        self.system = system
        self.message= []
        if self.system:
            self.message.append({"role": "system", "content": system})

    def __call__(self, message):
        """
        Calls the chatbot with a user message, appends it to the conversation history,
        and returns the assistant's response.
        """
        self.message.append({"role": "user", "content": message})
        result = self.execute()
        self.message.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        """
        Executes the language model with the current conversation history and returns the response.
        """
        llm = ChatGroq(model="Gemma2-9b-It")
        result = llm.invoke(self.message)
        return result.content


In [16]:
bot = Chatbot(system="You are a professional travel agent who always responds in bullet points.")
print(bot("What is the capital of France?"))

* Paris. 



In [17]:
print(bot("What is the capital of Iran?"))

* Tehran. 



In [13]:
bot.message

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user', 'content': 'What is the capital of France?'},
 {'role': 'assistant', 'content': 'The capital of France is **Paris**. 🗼  \n'},
 {'role': 'user', 'content': 'What is the capital of Iran?'},
 {'role': 'assistant', 'content': 'The capital of Iran is **Tehran**.  🇮🇷 \n'}]

# 2. We need to define prompt and this is where Agent becomes ReAct

In [6]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop your output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.


Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Python
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris

Please Note: if you get basic conversation questions like "hi","hello","how are you?",\n
you have to answer "hi","hello","i am good".
""".strip()

# 3. Let's give it some tools/functionalities based on prompt

*Understand the Regular expression*
Pattern Breakdown:

^: This matches the start of a string. It means the string must begin with what follows.

Action:: This is a literal match. It means the string must have the text "Action:" at the beginning.

(\w+):
The parentheses () define a capture group. This allows you to extract part of the string that matches this section.
\w+ matches one or more word characters (letters, digits, and underscores). This will capture a word that follows "Action:".

:: This is a literal colon that separates the word matched by (\w+) from the rest of the string.

(.*):
This is another capture group, where .* matches any character (.) zero or more times (*), which means it captures everything that comes after the second colon.

*What does it do?*
This regex is looking for a string that:

- Starts with the text "Action:".
- Has a word right after it, separated by a colon.
- Then, after another colon, it captures everything that follows.

In [7]:
import re
action_re = re.compile(r'^Action: (\w+): (.*)')


In [8]:
import httpx
def wikipedia(query):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    })
    return response.json()["query"]["search"][0]["snippet"]

In [9]:
import httpx
def simon_blog_search(query):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """
        select
          blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
          blog_entry.created
        from
          blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
          blog_entry_fts match escape_fts(:q)
        order by
          blog_entry_fts.rank
        limit
          1
        """.strip(),
        "_shape": "array",
        "q": query,
    })
    return response.json()[0]["text"]

In [10]:
def calculate(number):
    return eval(number)

# 4. Now we create the react loop

In [11]:
known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
    "simon_blog_search": simon_blog_search
}

In [29]:
def query(question,max_turns=5):
    """Run the agent loop for a given question."""
    i = 0
    bot = Chatbot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result

In [22]:
query("hi")

Okay, I understand the loop and my available actions. I'm ready to receive a question!  

Actions found: []


"Okay, I understand the loop and my available actions. I'm ready to receive a question!  \n"

In [23]:
query("how are you?")

Understood. I will follow the Thought, Action, PAUSE, Observation loop and provide an Answer based on the information gathered. 

Let's begin!  What is your first question? 


Actions found: []


"Understood. I will follow the Thought, Action, PAUSE, Observation loop and provide an Answer based on the information gathered. \n\nLet's begin!  What is your first question? \n\n"

In [24]:
query("one hundred * twelve")

Thought: I should calculate 100 * 12 
Action: calculate: 100 * 12
PAUSE 

Actions found: [<re.Match object; span=(0, 27), match='Action: calculate: 100 * 12'>]
 -- running calculate 100 * 12
Observation: 1200
Answer: 1200
Actions found: []


'Answer: 1200'

In [25]:
query("Has Simon Willison written about Python?")

Thought: I can check Simon's blog for mentions of Python. 

Action: simon_blog_search: Python
PAUSE  

Actions found: [<re.Match object; span=(0, 33), match='Action: simon_blog_search: Python'>]
 -- running simon_blog_search Python
Observation: The YDN Python Developer Center: I recently had the opportunity to put together the Python Developer Center for the Yahoo! Developer Network. YDN is one of my favourite parts of Yahoo! so I jumped at the chance, and the resulting mini-site is now online (YDN blog post here).

The bulk of the content is the HOWTOs, which discuss ways of accessing Yahoo! APIs using Python:


  Make Yahoo! Web Service REST calls with Python
  Cache API calls using Python
  Parse JSON using Python
  Parse XML using Python
  Access the Yahoo! Search APIs using pYsearch
  Access Yahoo! RSS feeds using Python


I had a lot of fun playing around with different ways of accessing the APIs and working out which ones were the most natural fit. The HOWTOs use urllib, urllib2

'Answer: Yes, Simon Willison has written about Python on his blog. \n'

In [31]:
query("What's the capital of Iran?")

Thought: I should look up Iran on Wikipedia to find the capital.

Action: wikipedia: Iran 
PAUSE 

 -- running wikipedia Iran 
Observation: display the Persian text in this article correctly. <span class="searchmatch">Iran</span>, officially the Islamic Republic of <span class="searchmatch">Iran</span> (IRI) and also known as Persia, is a country in West
Thought:  The observation confirms Iran is a country, but doesn't explicitly state its capital. I should continue looking for that information.


Action: wikipedia: Iran 
PAUSE 

 -- running wikipedia Iran 
Observation: display the Persian text in this article correctly. <span class="searchmatch">Iran</span>, officially the Islamic Republic of <span class="searchmatch">Iran</span> (IRI) and also known as Persia, is a country in West
Thought: I need to be more specific in my Wikipedia search. I'll look for the capital of Iran. 

Action: wikipedia: capital of Iran
PAUSE 

 -- running wikipedia capital of Iran
Observation: marks, boxes, o

'Answer: The capital of Iran is Tehran. \n'

In [32]:
query("What's the population of Tehran multiply by Toronto?")

Thought: I need to find the population of Tehran and Toronto, then multiply them together. 

Action: wikipedia: Population of Tehran
PAUSE 

 -- running wikipedia Population of Tehran
Observation: its Central District. With a <span class="searchmatch">population</span> <span class="searchmatch">of</span> around 9.8 million in the city, and 16.8 million in the metropolitan area, <span class="searchmatch">Tehran</span> is the most populous city in
Thought: I need the population of Toronto too. 

Action: wikipedia: Population of Toronto 
PAUSE 

 -- running wikipedia Population of Toronto 
Observation: <span class="searchmatch">Toronto</span> is the most populous city in Canada and the capital city <span class="searchmatch">of</span> the Canadian province <span class="searchmatch">of</span> Ontario. With a <span class="searchmatch">population</span> <span class="searchmatch">of</span> 2,794,356 in 2021, it is the
Thought: Now I have the populations of both cities, I can multiply them.

Ac

'Answer: The population of Tehran multiplied by the population of Toronto is 27.384688800000003 \n'